In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns
%matplotlib inline

In [ ]:
df_calendar = pd.read_csv('/Users/rwalters/Documents/Airbnb Data 2020/calendar.csv')
df_listings = pd.read_csv('/Users/rwalters/Documents/Airbnb Data 2020/listings.csv', dtype='str')
df_reviews = pd.read_csv('/Users/rwalters/Documents/Airbnb Data 2020/reviews.csv')

In [ ]:
# show listings data sample
df_listings.head()

In [ ]:
df_listings.shape

In [ ]:
# show reviews data sample
df_reviews.head()

In [ ]:
df_reviews.shape

In [ ]:
# show calendar data sample
df_calendar.head()

In [ ]:
df_calendar.isnull().sum()

In [ ]:
df_calendar.dtypes

In [ ]:
def change_to_float(df):
    df = df.str.replace(',', '')
    return df.str.replace('$', '').astype('float64')

In [ ]:
def change_to_int(df):
    return df.astype('int64')

In [ ]:
df_calendar['price'] = change_to_float(df_calendar['price'])
df_calendar['adjusted_price'] = change_to_float(df_calendar['adjusted_price'])

In [ ]:
df_calendar['price'] = change_to_int(df_calendar['minimum_nights'])
df_calendar['adjusted_price'] = change_to_int(df_calendar['maximum_nights'])

In [ ]:
df_calendar['date'] = df_calendar['date'].astype('datetime64[ns]')

In [ ]:
df_calendar.dtypes

In [ ]:
df_calendar.describe()

In [ ]:
date_count_df = df_calendar.groupby('date').size().reset_index(name='count')